In [ ]:
# Reads the .xml files and create a .csv with only the needed information 
# Uses the AwardTitle and the AbstractNarration information to create the dataset

import os, glob
import csv

# Get the data from the .xml tags
def filterData(data, tag):
    startTag = "<%s>" % (tag)
    endTag = "</%s>" % (tag)
    start = data.find(startTag) + len(startTag)
    end = data.find(endTag)
    return data[start:end]

#Clean the data removing unnecessary tag
def cleanData(data):
    startTag = "<![CDATA["
    endTag = "]]>"
    if startTag in data:
        data = data[len(startTag):-len(endTag)]
    return data

csvfile = open('abstract_narrations.csv', 'w')
writer = csv.writer(csvfile)
writer.writerow(['title', 'narration'])

emptyTags = ["<AwardTitle/>", "<AbstractNarration/>"]

for filename in glob.glob('files/*.xml'):
    with open(os.path.join(os.getcwd(), filename), 'r', encoding='UTF8') as f: # open in readonly mode
        data = f.read()
        if all(x not in data for x in emptyTags):
            
            title = filterData(data,"AwardTitle")
            title = cleanData(title)
            
            narration = filterData(data,"AbstractNarration")
            narration = cleanData(narration)
            
            writer.writerow([title, narration])
csvfile.close()